In [1]:
from langchain.document_loaders import PyMuPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA



In [2]:
def load_pdf(data_folder):
    loader = DirectoryLoader(data_folder, glob="*.pdf", loader_cls=PyMuPDFLoader)
    documents = loader.load()
    text_documents = [doc for doc in documents if doc.page_content]
    return text_documents


In [3]:
extracted_data = load_pdf("data/")

In [4]:
def text_split(text_documents):
    text_chunks = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    for doc in text_documents:
        chunks = text_splitter.split_documents([doc])
        text_chunks.extend(chunks)
    return text_chunks


In [5]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 1033


In [6]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [7]:
embeddings = download_hugging_face_embeddings()


c:\Users\Admin\anaconda3\envs\alchatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\anaconda3\envs\alchatbot\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
texts = [t.page_content for t in text_chunks]
faiss_index = FAISS.from_texts(texts, embeddings)

In [9]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q5_1.bin",
    model_type="llama",
    config={'max_new_tokens': 512, 'temperature': 0.8}
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=faiss_index.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [10]:
while True:
    user_input = input("Input Prompt: ")
    if user_input == 'Q':
        break
    result = qa({"query": user_input})
    print("Response:", result["result"])


c:\Users\Admin\anaconda3\envs\alchatbot\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response: A queue is a data structure that stores elements in a first-in-first-out (FIFO) order. It allows for efficient management of resources, such as printers or CPU time, by allowing multiple items to be added to the queue and then processed one at a time in the order they were added.
Response: The main difference between lists and arrays is that lists are mutable (can be changed), while arrays are immutable (cannot be changed). Additionally, lists can contain nested structures, such as other lists, while arrays are fixed in size and cannot have nested structures within them.
Response: Fibonacci is a sequence of numbers where each number is the sum of the two preceding numbers, starting from 0 and 1. The sequence begins like this: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, ... (sequence goes on indefinitely).
Response: Decision trees are a type of supervised learning algorithm used in machine learning to classify data based on its features. They work by recursively partitioning